In [2]:
import pandas as pd

df = pd.read_parquet('c:/A1/new_subway1.parquet')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58799739 entries, 0 to 58799738
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   transit_timestamp   object 
 1   station_complex_id  object 
 2   station_complex     object 
 3   borough             object 
 4   ridership           int16  
 5   latitude            float32
 6   longitude           float32
dtypes: float32(2), int16(1), object(4)
memory usage: 2.3+ GB


,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude
0,06/09/2023 12:00:00 AM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,4,40.632835,-73.947639
1,05/03/2023 12:00:00 PM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,58,40.632835,-73.947639
2,08/20/2023 03:00:00 PM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,12,40.632835,-73.947639
3,06/06/2023 11:00:00 PM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,1,40.632835,-73.947639
4,08/10/2023 11:00:00 AM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,14,40.632835,-73.947639


In [3]:
# Convert object types to category
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

# Check the data types to confirm changes
print(df.dtypes)

transit_timestamp     category
station_complex_id    category
station_complex       category
borough               category
ridership                int16
latitude               float32
longitude              float32
dtype: object


In [8]:
df.drop(columns=['borough', 'station_complex'], inplace=True)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58799739 entries, 0 to 58799738
Data columns (total 5 columns):
 #   Column              Dtype   
---  ------              -----   
 0   transit_timestamp   category
 1   station_complex_id  category
 2   ridership           int16   
 3   latitude            float32 
 4   longitude           float32 
dtypes: category(2), float32(2), int16(1)
memory usage: 785.7 MB


In [9]:
df.head()

,transit_timestamp,station_complex_id,ridership,latitude,longitude
0,06/09/2023 12:00:00 AM,359,4,40.632835,-73.947639
1,05/03/2023 12:00:00 PM,359,58,40.632835,-73.947639
2,08/20/2023 03:00:00 PM,359,12,40.632835,-73.947639
3,06/06/2023 11:00:00 PM,359,1,40.632835,-73.947639
4,08/10/2023 11:00:00 AM,359,14,40.632835,-73.947639


In [10]:
df['transit_timestamp'] = df['transit_timestamp'].str.split(' ').str[0]
df.head()

,transit_timestamp,station_complex_id,ridership,latitude,longitude
0,06/09/2023,359,4,40.632835,-73.947639
1,05/03/2023,359,58,40.632835,-73.947639
2,08/20/2023,359,12,40.632835,-73.947639
3,06/06/2023,359,1,40.632835,-73.947639
4,08/10/2023,359,14,40.632835,-73.947639


In [14]:
df.drop(columns=['latitude', 'longitude'], inplace=True)

In [15]:
df.groupby(['transit_timestamp', 'station_complex_id'], as_index=False)['ridership'].sum()

C:\Users\Kevin\AppData\Local\Temp\ipykernel_3736\3439217724.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['transit_timestamp', 'station_complex_id'], as_index=False)['ridership'].sum()


,transit_timestamp,station_complex_id,ridership
0,01/01/2023,1,3848
1,01/01/2023,10,12783
2,01/01/2023,100,745
3,01/01/2023,101,4313
4,01/01/2023,103,2596
...,...,...,...
363795,12/31/2023,97,4909
363796,12/31/2023,98,2385
363797,12/31/2023,99,1496
363798,12/31/2023,TRAM1,6064


In [1]:
import pandas as pd
df = pd.read_parquet('c:/A1/ready_subway.parquet')

In [4]:
df.tail()

,transit_timestamp,station_complex_id,ridership
58799734,05/13/2023,359,3
58799735,03/27/2023,359,1
58799736,01/27/2023,359,1
58799737,05/29/2023,359,7
58799738,01/17/2023,359,65


In [2]:
subway = pd.read_csv('c:/A3/MTA_Subway_Stations.csv')
subway = subway.rename(columns={
    'Complex ID': 'station_complex_id',
    'GTFS Longitude': 'longitude',
    'GTFS Latitude': 'latitude'
})

In [10]:
import geopandas as gpd
from shapely.geometry import Point

# Load the location coordinates file as GeoDataFrame
location_coords_gdf = gpd.read_file('c:/A3/subway.shp')
print(location_coords_gdf.head())
subway = location_coords_gdf[['', 'longitude', 'latitude']]

grouped_with_coords = pd.merge(df, subway, on='station_complex_id', how='left')

grouped_with_coords['geometry'] = grouped_with_coords.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
df = gpd.GeoDataFrame(grouped_with_coords, geometry='geometry')
df.set_crs(epsg=4326, inplace=True)


nta_gdf = gpd.read_file('c:/A/nyct2020_24b/nyct2020_24b/nyct2020.shp')

# Perform spatial join to find the corresponding NTA for each pickup location
result_gdf = gpd.sjoin(df, nta_gdf, how='left', op='intersects')

# Now your result_gdf contains the NTA information
print(result_gdf.head())

          id name  objectid rt_symbol    shape_len  \
0  2000393.0    G     753.0         G  2438.200249   
1  2000394.0    G     754.0         G  3872.834411   
2  2000469.0    Q     755.0         N  1843.366331   
3  2000294.0    M     756.0         B  1919.559203   
4  2000296.0    M     757.0         B  2385.698536   

                                 url  \
0  http://web.mta.info/nyct/service/   
1  http://web.mta.info/nyct/service/   
2  http://web.mta.info/nyct/service/   
3  http://web.mta.info/nyct/service/   
4  http://web.mta.info/nyct/service/   

                                            geometry  
0  LINESTRING (-73.99488 40.68020, -73.99427 40.6...  
1  LINESTRING (-73.97958 40.65993, -73.97966 40.6...  
2  LINESTRING (-73.97586 40.57597, -73.97654 40.5...  
3  LINESTRING (-73.92414 40.75229, -73.92405 40.7...  
4  LINESTRING (-73.91345 40.75617, -73.90905 40.7...  


KeyError: "None of [Index(['', 'longitude', 'latitude'], dtype='object')] are in the [columns]"